
# SIP Lab 3 : Discrete cosine transform, Introduction to JPEG.

### Basile Mouret, Malik Hacini

## Exercice 1


### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

## Exercice 2

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5